In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey

# urls
places_api = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"

# data paths
city_data_path = "../output_data/cities.csv"

In [ ]:
city_data = pd.read_csv(city_data_path)

city_data

In [ ]:
# gmaps configuration
gmaps.configure(api_key=gkey)

In [ ]:
# function for heatmap1
def heatmap1(locations, weights):
    fig = gmaps.figure()
    heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False,)
    fig.add_layer(heat_layer) 
    fig   
    return fig

# function for heatmap2
def heatmap2(humidity_locations, hotel_locations, humidity, hotel_info, hover_text):
    fig = gmaps.figure()
    
    heat_layer = gmaps.heatmap_layer(humidity_locations, weights=humidity, dissipating=False,)
    marker_layer = gmaps.marker_layer(hotel_locations, hover_text=hover_text)
    
    fig.add_layer(heat_layer) 
    fig.add_layer(marker_layer)
    
    fig  
    
    return fig

In [ ]:
locations = city_data[["Lat", "Lng"]]
weights = city_data["Humidity"]
heatmap1(locations, weights)

In [ ]:
vacation_cities = city_data.dropna()

cold = vacation_cities.loc[vacation_cities["Max Temp"] < 65].index
vacation_cities = vacation_cities.drop(cold)

hot = vacation_cities.loc[vacation_cities["Max Temp"] > 85].index
vacation_cities = vacation_cities.drop(hot)

windy = vacation_cities.loc[vacation_cities["Wind Speed"] > 10].index
vacation_cities = vacation_cities.drop(windy)

cloudy = vacation_cities.loc[vacation_cities["Cloudiness"] > 0].index
vacation_cities = vacation_cities.drop(cloudy)

humid = vacation_cities.loc[vacation_cities["Humidity"] > 60].index
vacation_cities = vacation_cities.drop(humid)

hotel_df = vacation_cities

hotel_df

In [ ]:
hotel_list = []
address_list = []
term = "Hotel"
inputtype = "textquery"
radius = "10"
fields = ["name", "formatted_address"]

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    locationbias = f"circle:{radius}@{lat},{lng}"
    target_url = f"{places_api}key={gkey}&input={term}&inputtype={inputtype}&locationbias=circle:{radius}@{lat},{lng}&fields=name,formatted_address"
    response = requests.get(target_url).json()
    hotel_list.append(response["candidates"][0]["name"])
    address_list.append(response["candidates"][0]["formatted_address"])

hotel_df["Hotel Name"] = hotel_list
hotel_df["Hotel Address"] = address_list

hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
humidity_locations = city_data[["Lat", "Lng"]]
weight = city_data["Humidity"]
hover_text = hotel_list

# Display Map
heatmap2(humidity_locations, hotel_locations, weight, hotel_info, hover_text)